In [1]:
## STEP 1: LOAD DEPENDENCIES, SET GLOBAL PARAMETERS, DEFINE FUNCTIONS

# Import Libraries and Library Settings
import numpy as np
import scipy.io as sio
from os.path import dirname, join as pjoin
import time
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib tk
from matplotlib.animation import FuncAnimation
import glob
import os
import re
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Setting Params for plotting & troubleshooting
np.set_printoptions(threshold=np.inf)
plt.rcParams['animation.html']='jshtml'

# Define Functions in Use

# Split a string to determine the integer iteration, usually to find the next file (largest number)
def func(p):
    return int(((p.split('.')[0]).split('_'))[-1])

# For a histogram plotted via grid, add labels to the top of each bar
def addlabels(x,y):
    x2 = x[:-1]
    for i in range(len(x2)):
        ax4.annotate(y[i],(x2[i]+(x[1]-x[0])/2,y[i]+5),ha='center',color='r',fontsize=9)

In [2]:
## STEP 2: INITIALIZE CHANGING VARIABLES

piece_no = 0
time_win = 20 #The amount of time (ms) you want to view in each frame 
spike_t = 3 #How many standard deviations above noise to find spikes
real_time_win = 40 #The amount of time (ms) you want to view in the current buffer
live = False # True / False
non_live_datarun = 9 # If looking at old data, enter in the datarun number
non_live_date_piece = '2022-01-14-0' # If looking at old data, enter in the date & piece
save = False # The resulting noise matrix and noise values, True / False

In [3]:
# For current variable names
path = r'/Shakedown/Users/vision/Desktop/Xilinx'
date_piece = '2022-01-14-0'
datarun = 'data009'
buffer_num = 19

string = 'gmem1'

file_next = pjoin(date_piece,datarun,datarun+'_'+str(buffer_num)+'.mat')

## STEP 6: CONVERT & PROCESS DATA 

# Load the next buffer file
mat_contents = sio.loadmat(file_next)
data = mat_contents[string][0][:]

In [ ]:
# Initialize Figure and Properties
# Figure
plt.close('all')
fig = plt.figure(figsize=(10,10),facecolor='white',constrained_layout=False)
fig.show()
plt.style.use('fivethirtyeight')

sample_win = int(time_win/0.05)
fig_rows_pre = 0

# Initialize Data Conversion & Analysis Variables
chan_num_pre = -1 #Setting the previous electrode in the count/time determination into an impossibility
buffer_num = 0 #Setting the first buffer incoming b/c we will be plotting all incoming buffers

noise_mean = np.zeros((32,32))
noise_std = np.zeros((32,32))
noise_cnt = np.zeros((32,32))
spike_avg = np.zeros((32,32))
spike_std = np.zeros((32,32))
spike_cnt = np.zeros((32,32))

In [ ]:
for k in range(0,50):

    for i in range(0,1024):

    #     ch = i

    #     ch_row_y = (ch/32).astype(int)
    #     ch_col_x = int(ch - ch_row_y*32)

        # Initialize Variables
        data_real = np.zeros((32,32,len(data)-2))  #Initialize to max possible length. Note: Throw out first two values b/c garbo
        cnt_real = np.zeros((32,32,len(data)-2))
        cnt_pre = 0 #Check for cnt changes, double cnt
        chan_index_pre = 1025 #Check for chan changes, double cnt
        N = 0 # Sample times (DOES NOT ALLOW NON-COLLISION FREE SAMPLES)

        for i in range(2,len(data)-1): 
            # Convert bit number into binary
            word = (np.binary_repr(data[i],32))

            # Break that binary into it's respective pieces and convert to bit number
            cnt = int(word[12:14],2)
            col = int(word[27:32],2)
            row = int(word[22:27],2)
            chan_index = row*32 + col

            # Only record the unique non-double count sample
            if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):

                # Sample time only changes when cnt changes
                if cnt != cnt_pre:
                    N += 1

                # On the occurance the first cnt is not 0, make sure sample time is 0
                if i == 2:
                    N = 0

                # Update variables
                cnt_pre = cnt
                chan_index_pre = chan_index

                # Record pertinent data
                data_real[row][col][N] = int(word[14:22],2)
                cnt_real[row][col][N] = cnt

        # Truncate data to be the true number of samples       
        num_samples = N
        data_real = data_real[:,:,:N]
        cnt_real = cnt_real[:,:,:N]

        total_time = num_samples*0.05
        times = np.linspace(0,total_time,num_samples)

        num_sam = np.zeros((32,32,1))
        avg_val = np.zeros((32,32,1))

        num_sam[:,:,0] = np.count_nonzero(data_real,axis=2)

        fig_rows = np.count_nonzero(num_sam)
        ch_ind = np.zeros((fig_rows))
        
        x2 = np.zeros((fig_rows,1,0))
        y2 = np.zeros((fig_rows,1,0))
        tempx = np.zeros((fig_rows,1,sample_win))
        tempy = np.zeros((fig_rows,1,sample_win))
        axs_name = []*fig_rows
        
        # AX2) Initialize Time Domain Subplots
        for k in range(fig_rows):
            axs = fig.add_subplot(ax2[k,0])
            axs_name = np.append(axs_name,axs)
            text_elec = 'Elec # ' + str(fig_ind[k])
            axs_name[k].annotate(text_elec,(0.01,0.9),xycoords='axes fraction',va='center',fontsize = 11,color='blue')
            
    




        
        incom_cnt = num_sam[:,:,0] 

        # AX2) Time Domain Electrodes to Plot
        fig_rows = np.count_nonzero(num_sam)
        if fig_rows == 0:
            fig_rows = 1
        if fig_rows > 4:
            fig_rows = 4
        fig_elec = np.zeros((fig_rows,2))

        fig_ind = np.argsort(num_sam.flatten())[-fig_rows:]
        fig_elec[:,0] = (fig_ind[:] / 32).astype(int)
        fig_elec[:,1] = fig_ind[:] - (fig_elec[:,0]*32)
        fig_elec = fig_elec.astype(int)

        # AX1,AX3,AX4) Finding average ADC of samples per electrode
        mask = np.copy(data_real)
        mask[mask==0] = np.nan
        avg_val[:,:,0] = np.nanmean(mask,axis=2)
        avg_val = np.nan_to_num(avg_val,nan=0)
        incom_mean = avg_val[:,:,0]


    
    
    
    
    
    
    